# Задание 2 (средний pFound)

## pFound
Исходные данные - Yandex Cup 2022 Analytics
- Ссылка - https://yandex.ru/cup/analytics/analysis/ , пример A. Рассчитать pFound
- Данные - https://yadi.sk/d/guqki4UI4hFlXQ
- Формула
$$pFound@K = \sum_{i=1}^{k} pLook[i]\ pRel[i]$$

$$pLook[1] = 1$$

$$pLook[i] = pLook[i-1]\ (1 - pRel[i-1])\ (1 - pBreak)$$

$$pBreak = 0.15$$

**Задача** - написать функцию, которая принимает на вход dataframe (после join), а на выходе дает средний pFound по всем query.
- Запрещается использовать циклы for для расчет метрики (как полностью, так и ее частей).
- Усложнение, если задача показалась легкой - попробуйте обойтись без groupby (не уверен, что это возможно, но вдруг вы справитесь)

# Импорты

In [1]:
import warnings
import pandas as pd
import requests

from urllib.parse import urlencode

In [2]:
# Disable warnings
warnings.filterwarnings('ignore')

# Загрузка данных

In [7]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://yadi.sk/d/guqki4UI4hFlXQ'

# Получаем загрузочную ссылку
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

# Загружаем файл и сохраняем его
download_response = requests.get(download_url)
with open('data.zip', 'wb') as f: 
    f.write(download_response.content)

In [8]:
!unzip data.zip
!rm hidden_task.zip
!unzip open_task.zip

Archive:  data.zip
  inflating: hidden_task.zip         
  inflating: open_task.zip           
Archive:  open_task.zip
   creating: open_task/
  inflating: open_task/qid_query.tsv  
  inflating: open_task/hostid_url.tsv  
  inflating: open_task/qid_url_rating.tsv  


# Решение Yandex Cup 2022 Analytics

In [3]:
# считываем данные
qid_query = pd.read_csv("open_task/qid_query.tsv", sep="\t", names=["qid", "query"])
qid_url_rating = pd.read_csv("open_task/qid_url_rating.tsv", sep="\t", names=["qid", "url", "rating"])
hostid_url = pd.read_csv("open_task/hostid_url.tsv", sep="\t", names=["hostid", "url"])

# делаем join двух таблиц, чтобы было просто брать url с максимальным рейтингом
qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")

In [4]:
def plook(ind, rels):
    if ind == 0:
        return 1
    return plook(ind-1, rels)*(1-rels[ind-1])*(1-0.15)


def pfound(group):
    max_by_host = group.groupby("hostid")["rating"].max() # максимальный рейтинг хоста
    top10 = max_by_host.sort_values(ascending=False)[:10] # берем топ10 урлов с наивысшим рейтингом
    pfound = 0
    for ind, val in enumerate(top10):
        pfound += val*plook(ind, top10.values)
    return pfound


qid_pfound = qid_url_rating_hostid.groupby('qid').apply(pfound) # группируем по qid и вычисляем pfound
qid_max = qid_pfound.idxmax() # берем qid с максимальным pfound

qid_query[qid_query["qid"] == qid_max]

,qid,query
12,295761,гугл переводчик


In [5]:
qid_pfound

qid
10387     0.497771
20860     0.655448
21070     0.497771
35618     0.437794
107538    0.354808
150126    0.366109
168170    0.481255
176370    0.393661
192007    0.191170
213932    0.347005
221830    0.497771
242953    0.497771
253476    0.497771
295761    0.900836
346214    0.263596
347852    0.618534
360100    0.470204
366042    0.309314
375608    0.497771
380923    0.429989
dtype: float64

In [6]:
# Средний pFound
qid_pfound.mean()

0.4603173929969002

# Решение без цикла

## Функция для рассчета pFound

In [19]:
def pFound(df: pd.DataFrame, 
                K: int = 10, 
                pBreak: float = 0.15):
    
    merged_df = df.copy()
    
    # Удаляем дубликаты взаимодейтсвий 
    duplicates = merged_df[merged_df.duplicated()]
    merged_df = merged_df.drop_duplicates(keep='first')
    
    # Когда у запроса есть несколько документов с одним и тем же hostid: 
    # 1) оставить только максимально релевантный документ,
    # 2) если несколько документов максимально релевантны, выбрать любой.
    merged_df = merged_df.groupby(['qid', 'hostid'])['rating'].max().reset_index()
    
    # Для каждого запроса берем топ K (K = 10) hostid с максимальными рейтингами, сортируем по убыванию
    top_K = merged_df.sort_values(['qid', 'rating'], ascending=False).groupby(['qid']).head(K)
    
    # Порядковые номера для hostid внутри каждого qid (0: k - 1)
    top_K['count'] = top_K.groupby('qid').cumcount()

    # Вспомогательные множители для расчета pLook
    top_K['1-Rel'] = (1 - top_K['rating']).shift(1) # (1 - Rel)
    top_K['1-pBreak'] = 1 - pBreak # (1 - pBreak)
    
    # pLook[1] = 1
    top_K.loc[top_K['count'] == 0, ['1-Rel', '1-pBreak']] = 1

    # interim - вспомогательная колонка для расчета pLook
    top_K['interim'] = top_K['1-Rel'] * top_K['1-pBreak']
    top_K['pLook'] = top_K.groupby('qid')['interim'].cumprod()
    
    # pFound@K
    top_K['pFound'] = top_K['pLook'] * top_K['rating']
    pFound_qid = top_K.groupby('qid')['pFound'].sum()
    
    return pFound_qid

In [20]:
print("Yandex pfound")
print(qid_pfound, '\n')
print("Custom pFound")
print(pFound(qid_url_rating_hostid))

Yandex pfound
qid
10387     0.497771
20860     0.655448
21070     0.497771
35618     0.437794
107538    0.354808
150126    0.366109
168170    0.481255
176370    0.393661
192007    0.191170
213932    0.347005
221830    0.497771
242953    0.497771
253476    0.497771
295761    0.900836
346214    0.263596
347852    0.618534
360100    0.470204
366042    0.309314
375608    0.497771
380923    0.429989
dtype: float64 

Custom pFound
qid
10387     0.497771
20860     0.655448
21070     0.497771
35618     0.437794
107538    0.354808
150126    0.366109
168170    0.481255
176370    0.393661
192007    0.191170
213932    0.347005
221830    0.497771
242953    0.497771
253476    0.497771
295761    0.900836
346214    0.263596
347852    0.618534
360100    0.470204
366042    0.309314
375608    0.497771
380923    0.429989
Name: pFound, dtype: float64


Вывод pFound аналогичен результату qid_pfound Яндекса.

## Функция для рассчета среднего pFound

In [17]:
def pFound(df: pd.DataFrame, 
                K: int = 10, 
                pBreak: float = 0.15):
    
    merged_df = df.copy()
    
    # Удаляем дубликаты взаимодейтсвий 
    duplicates = merged_df[merged_df.duplicated()]
    merged_df = merged_df.drop_duplicates(keep='first')
    
    # Когда у запроса есть несколько документов с одним и тем же hostid: 
    # 1) оставить только максимально релевантный документ,
    # 2) если несколько документов максимально релевантны, выбрать любой.
    merged_df = merged_df.groupby(['qid', 'hostid'])['rating'].max().reset_index()
    
    # Для каждого запроса берем топ K (K = 10) hostid с максимальными рейтингами, сортируем по убыванию
    top_K = merged_df.sort_values(['qid', 'rating'], ascending=False).groupby(['qid']).head(K)
    
    # Порядковые номера для hostid внутри каждого qid (0: k - 1)
    top_K['count'] = top_K.groupby('qid').cumcount()

    # Вспомогательные множители для расчета pLook
    top_K['1-Rel'] = (1 - top_K['rating']).shift(1) # (1 - Rel)
    top_K['1-pBreak'] = 1 - pBreak # (1 - pBreak)
    
    # pLook[1] = 1
    top_K.loc[top_K['count'] == 0, ['1-Rel', '1-pBreak']] = 1

    # interim - вспомогательная колонка для расчета pLook
    top_K['interim'] = top_K['1-Rel'] * top_K['1-pBreak']
    top_K['pLook'] = top_K.groupby('qid')['interim'].cumprod()
    
    # pFound@K
    top_K['pFound'] = top_K['pLook'] * top_K['rating']
    pFound_qid = top_K.groupby('qid')['pFound'].sum()
    pFound_qid_mean = pFound_qid.mean()
    
    return pFound_qid_mean

In [18]:
print("Yandex mean pfound")
print(qid_pfound.mean(), '\n')
print("Custom mean pFound")
print(pFound_mean(qid_url_rating_hostid))

Yandex mean pfound
0.4603173929969002 

Custom mean pFound
0.4603173929969002


Вывод pFound_mean аналогичен результату qid_pfound.mean() Яндекса.

In [11]:
%timeit pFound_mean(qid_url_rating_hostid)

7.82 ms ± 222 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
